# SKT 한전조수 가공요금 자동 정산화

In [1]:
import os
import pandas as pd
import sqlite3

import xlsxwriter
import openpyxl
from openpyxl.workbook import Workbook
from openpyxl.styles import Font, Fill

import warnings
warnings.filterwarnings('ignore')

In [2]:
# DB 경로
sqlite = 'd:/dev/SQL_DB/한전조수DB.db'

# splite 파일 불러오기
def open_sqlite(sqlite_file, sql_str):
    conn = sqlite3.connect(sqlite_file)
    ret = pd.read_sql(sql_str, conn)
    conn.close()
    return ret 

def save_sqlite(sqlite_file, table_name ,df):
    conn = sqlite3.connect(sqlite_file)
    df.to_sql(table_name, conn,if_exists='append')
    conn.close()

In [3]:
print('◎ 전월데이터 년월을 입력해주세요 (예제 : 201901)')
month_before = input()
print('==> 전월데이터는 {0}으로 입력했습니다.'.format(month_before))
print('◎ 당월데이터 년월을 입력해주세요 (예제 : 201901)')
month_now = input()
print('==> 당월데이터는 {0}으로 입력했습니다.'.format(month_now))

◎ 전월데이터 년월을 입력해주세요 (예제 : 201901)


 201905


==> 전월데이터는 201905으로 입력했습니다.
◎ 당월데이터 년월을 입력해주세요 (예제 : 201901)


 201906


==> 당월데이터는 201906으로 입력했습니다.


# SQLite 에서 가공요금표 테이블 불러오기 
* 기준은 입력한 month 기준

In [4]:
sql_str = "SELECT * FROM 가공요금표 WHERE month = '"+month_now+"'"

당월데이터 = open_sqlite(sqlite, sql_str)
당월데이터 = 당월데이터.drop(columns=['index'])

sql_str_1 = "SELECT * FROM 가공요금표 WHERE month = '"+month_before+"'"

전월데이터 = open_sqlite(sqlite, sql_str_1)
전월데이터 = 전월데이터.drop(columns=['index'])

In [5]:
당월데이터[당월데이터['한전본부'].isnull()]

,사업소코드,전산화번호,정상조수,정상요금,무단조수,무단요금,무단추징금(100%),신규위약조수,신규위약요금,무단추징금(200%),...,자진신고무단요금,자진신고무단추징금(150%),신규무단조수,신규무단요금,신규무단추징금(300%),한전본부,지사명,SKT부서명,ONS부서명,month


In [6]:
전월데이터[전월데이터['한전본부'].isnull()]

,사업소코드,전산화번호,정상조수,정상요금,무단조수,무단요금,무단추징금(100%),신규위약조수,신규위약요금,무단추징금(200%),...,자진신고무단요금,자진신고무단추징금(150%),신규무단조수,신규무단요금,신규무단추징금(300%),한전본부,지사명,SKT부서명,ONS부서명,month


# SKT 한전공가 이용료 자료 추출

In [7]:
#### 부서명 정렬순 맞춰주기 ####
당월데이터['SKT부서명'] = pd.Categorical(당월데이터['SKT부서명'],['수도권유선Infra','동부유선Infra','서부유선Infra','중부유선Infra'])

당월데이터['ONS부서명'] = pd.Categorical(당월데이터['ONS부서명'],['강남전송운용','강북전송운용','경남전송운용',
                                                  '경북전송운용','서부전송운용','충청전송운용','강원전송운용'])


#### 통합 전주 sheet 작업하기 ####
통합1 = pd.pivot_table(data = 당월데이터, index=['SKT부서명','ONS부서명','한전본부'], values=['정상조수', '정상요금', '무단조수', '무단요금', '무단추징금(100%)',
       '신규위약조수', '신규위약요금', '무단추징금(200%)', 'HFC무단조수', 'HFC무단요금',
       'HFC무단추징금(100%)', '접속개소조수', '접속개소요금', '접속개소추징금(100%)', '자진신고무단조수',
       '자진신고무단요금', '자진신고무단추징금(150%)', '신규무단조수', '신규무단요금', '신규무단추징금(300%)'], aggfunc=sum).reset_index()

통합1['전수(조수)'] = 통합1['정상조수']+통합1['무단조수']+통합1['신규위약조수']+통합1['HFC무단조수']+통합1['접속개소조수']+\
                    통합1['자진신고무단조수']+통합1['신규무단조수']

통합1['이용료(원)①'] = 통합1['정상요금']
통합1['면탈금(원)②'] = 통합1['무단요금']+통합1['신규위약요금']+통합1['HFC무단요금']+\
                    통합1['접속개소요금']+통합1['자진신고무단요금']+통합1['신규무단요금']
통합1['계③=①+②'] = 통합1['이용료(원)①'] + 통합1['면탈금(원)②']
통합1['무단전주본수'] = 통합1['무단조수']+통합1['신규위약조수']+통합1['HFC무단조수']+통합1['접속개소조수']+통합1['자진신고무단조수']+통합1['신규무단조수']
통합1['전주위약요금'] = 통합1['무단추징금(100%)']+통합1['무단추징금(200%)']+통합1['HFC무단추징금(100%)']+통합1['접속개소추징금(100%)']+통합1['자진신고무단추징금(150%)']+통합1['신규무단추징금(300%)']

통합 = 통합1.drop(columns=['HFC무단요금', 'HFC무단조수', 'HFC무단추징금(100%)',
       '무단요금', '무단조수', '무단추징금(100%)', '무단추징금(200%)', '신규무단요금', '신규무단조수',
       '신규무단추징금(300%)', '신규위약요금', '신규위약조수', '자진신고무단요금', '자진신고무단조수',
       '자진신고무단추징금(150%)', '접속개소요금', '접속개소조수', '접속개소추징금(100%)', '정상요금', '정상조수'])

#### 통합 전주 Sheet 엑셀 추출 ####
통합.to_excel('./가공엑셀자료/'+month_now+'_SKT한전공가이용료.xlsx', encoding='euc-kr')

# T1망 전주사용 세부현황

In [8]:
전주사용세부1 = pd.pivot_table(data = 당월데이터, index=['SKT부서명','ONS부서명','한전본부','지사명'], values=['정상조수', '정상요금', '무단조수', '무단요금', '무단추징금(100%)',
       '신규위약조수', '신규위약요금', '무단추징금(200%)', 'HFC무단조수', 'HFC무단요금',
       'HFC무단추징금(100%)', '접속개소조수', '접속개소요금', '접속개소추징금(100%)', '자진신고무단조수',
       '자진신고무단요금', '자진신고무단추징금(150%)', '신규무단조수', '신규무단요금', '신규무단추징금(300%)'], aggfunc=sum).reset_index()

전주사용세부 = 전주사용세부1[['SKT부서명', 'ONS부서명', '한전본부', '지사명','정상조수','정상요금','무단조수','무단요금','신규위약조수','신규위약요금','HFC무단조수','HFC무단요금','접속개소조수','접속개소요금','자진신고무단조수','자진신고무단요금','신규무단조수','신규무단요금','무단조수','무단요금','신규위약조수','신규위약요금','HFC무단조수','HFC무단요금','접속개소조수','접속개소요금','자진신고무단조수','자진신고무단요금','신규무단조수','신규무단요금']]

전주사용세부.to_excel('./가공엑셀자료/'+month_now+'_전주사용세부현황.xlsx', encoding='euc-kr')

# 전주조수 사용 현황

In [9]:
전주사용량 =  pd.pivot_table(data = 당월데이터, index=['SKT부서명','한전본부'], values=['정상조수', '무단조수', 
       '신규위약조수','HFC무단조수', '접속개소조수', '자진신고무단조수', '신규무단조수',], aggfunc=sum).reset_index()

전주사용량 = 전주사용량[['SKT부서명','한전본부','정상조수', '무단조수', '신규위약조수','HFC무단조수','접속개소조수','자진신고무단조수','신규무단조수']]

전주사용량['미승인(계)'] = 전주사용량['무단조수']+전주사용량['신규위약조수']+전주사용량['HFC무단조수']+전주사용량['접속개소조수']+전주사용량['자진신고무단조수']+전주사용량['신규무단조수']
전주사용량['전체(계)'] = 전주사용량['정상조수']+전주사용량['무단조수']+전주사용량['신규위약조수']+전주사용량['HFC무단조수']+전주사용량['접속개소조수']+전주사용량['자진신고무단조수']+전주사용량['신규무단조수']

전주사용량.to_excel('./가공엑셀자료/'+month_now+'_전주사용량.xlsx', encoding='euc-kr')

# 전월~당월 조수비교 테이블 만들기

In [10]:
당월조수비교 = pd.pivot_table(당월데이터, index=['한전본부','지사명'],
                        values=['정상조수', '무단조수', '신규위약조수', 'HFC무단조수','접속개소조수', '자진신고무단조수','신규무단조수'],
                        aggfunc=sum).reset_index()

전월조수비교 = pd.pivot_table(전월데이터, index=['한전본부','지사명'],
                        values=['정상조수', '무단조수', '신규위약조수', 'HFC무단조수','접속개소조수', '자진신고무단조수','신규무단조수'],
                        aggfunc=sum).reset_index()

당월조수비교 = 당월조수비교[['한전본부','지사명','정상조수','무단조수','신규위약조수','HFC무단조수','접속개소조수','자진신고무단조수','신규무단조수']]
전월조수비교 = 전월조수비교[['한전본부','지사명','정상조수','무단조수','신규위약조수','HFC무단조수','접속개소조수','자진신고무단조수','신규무단조수']]

당월조수비교.rename(columns={'정상조수' : '당월_1번',
                              '무단조수' : '당월_2번',
                              '신규위약조수' : '당월_3번',
                               'HFC무단조수' : '당월_4번',
                               '접속개소조수' : '당월_5번',
                               '자진신고무단조수' : '당월_6번',
                               '신규무단조수' : '당월_7번',
                      
                      }, inplace = True)

전월조수비교.rename(columns={'정상조수' : '전월_1번',
                              '무단조수' : '전월_2번',
                              '신규위약조수' : '전월_3번',
                               'HFC무단조수' : '전월_4번',
                               '접속개소조수' : '전월_5번',
                               '자진신고무단조수' : '전월_6번',
                               '신규무단조수' : '전월_7번',
                      
                      }, inplace = True)

조수비교 = pd.merge(당월조수비교, 전월조수비교, on = ['한전본부','지사명'], how='left')

조수비교['1번_Gap'] = 조수비교['당월_1번'] - 조수비교['전월_1번']
조수비교['2번_Gap'] = 조수비교['당월_2번'] - 조수비교['전월_2번']
조수비교['3번_Gap'] = 조수비교['당월_3번'] - 조수비교['전월_3번']
조수비교['4번_Gap'] = 조수비교['당월_4번'] - 조수비교['전월_4번']
조수비교['5번_Gap'] = 조수비교['당월_5번'] - 조수비교['전월_5번']
조수비교['6번_Gap'] = 조수비교['당월_6번'] - 조수비교['전월_6번']
조수비교['7번_Gap'] = 조수비교['당월_7번'] - 조수비교['전월_7번']

조수비교['당월_총조수'] = 조수비교['당월_1번']+조수비교['당월_2번']+조수비교['당월_3번']+조수비교['당월_4번']\
+조수비교['당월_5번']+조수비교['당월_6번']+조수비교['당월_7번']
조수비교['전월_총조수'] = 조수비교['전월_1번']+조수비교['전월_2번']+조수비교['전월_3번']+조수비교['전월_4번']\
+조수비교['전월_5번']+조수비교['전월_6번']+조수비교['전월_7번']

조수비교['당월_무단조수'] = 조수비교['당월_2번']+조수비교['당월_3번']+조수비교['당월_4번']\
+조수비교['당월_5번']+조수비교['당월_6번']+조수비교['당월_7번']
조수비교['전월_무단조수'] = 조수비교['전월_2번']+조수비교['전월_3번']+조수비교['전월_4번']\
+조수비교['전월_5번']+조수비교['전월_6번']+조수비교['전월_7번']

조수비교['총조수_Gap'] = 조수비교['당월_총조수']-조수비교['전월_총조수']
조수비교['무단조수_Gap'] = 조수비교['당월_무단조수']-조수비교['전월_무단조수']

조수비교_정렬 = 조수비교[['한전본부', '지사명','당월_1번','전월_1번','1번_Gap','당월_2번','전월_2번','2번_Gap','당월_3번','전월_3번','3번_Gap',
      '당월_4번','전월_4번','4번_Gap','당월_5번','전월_5번','5번_Gap','당월_6번','전월_6번','6번_Gap','당월_7번','전월_7번','7번_Gap',
      '당월_총조수','전월_총조수','총조수_Gap','당월_무단조수', '전월_무단조수', '무단조수_Gap']]

조수비교_정렬.to_excel('./가공엑셀자료/'+month_now+'_조수비교.xlsx', encoding='euc-kr')

# 전산화번호 기준의 증감내역 분석

In [11]:
당월데이터['총조수'] = 당월데이터['정상조수']+당월데이터['무단조수']+당월데이터['신규위약조수']+당월데이터['HFC무단조수']+\
                    당월데이터['접속개소조수']+당월데이터['자진신고무단조수']+당월데이터['신규무단조수']

당월데이터['총요금'] = 당월데이터['정상요금']+당월데이터['무단요금']+당월데이터['무단추징금(100%)']+당월데이터['신규위약요금']+\
                    당월데이터['무단추징금(200%)']+당월데이터['HFC무단요금']+당월데이터['HFC무단추징금(100%)']+\
당월데이터['접속개소요금']+당월데이터['접속개소추징금(100%)']+당월데이터['자진신고무단요금']+당월데이터['자진신고무단추징금(150%)']+\
                    당월데이터['신규무단요금']+당월데이터['신규무단추징금(300%)']

전월데이터['총조수'] = 전월데이터['정상조수']+전월데이터['무단조수']+전월데이터['신규위약조수']+전월데이터['HFC무단조수']+\
                    전월데이터['접속개소조수']+전월데이터['자진신고무단조수']+전월데이터['신규무단조수']

전월데이터['총요금'] = 전월데이터['정상요금']+전월데이터['무단요금']+전월데이터['무단추징금(100%)']+전월데이터['신규위약요금']+\
                    전월데이터['무단추징금(200%)']+전월데이터['HFC무단요금']+전월데이터['HFC무단추징금(100%)']+\
전월데이터['접속개소요금']+전월데이터['접속개소추징금(100%)']+전월데이터['자진신고무단요금']+전월데이터['자진신고무단추징금(150%)']+\
                    전월데이터['신규무단요금']+전월데이터['신규무단추징금(300%)']

In [12]:
전산화번호_세부비교 = pd.merge(당월데이터,전월데이터, on = ['전산화번호','사업소코드'], how='outer')

In [13]:
전산화번호_세부비교['조수_Gap'] = 전산화번호_세부비교['총조수_x'] - 전산화번호_세부비교['총조수_y']
전산화번호_세부비교['요금_Gap'] = 전산화번호_세부비교['총요금_x'] - 전산화번호_세부비교['총요금_y']


In [14]:
#조수가 0보다 큰거 : 조수 증가
#조수가 0보다 작은거 : 조수 감소
#요금이 0보다 큰거 : 요금 증가
#요금이 0본다 작은거 : 요금 감소

#당월(x)에만 데이터가 있을경우 : 신규
#전월(y)에만 데이터가 있을경우 : 해지 

In [15]:
조수증가세부 = 전산화번호_세부비교[전산화번호_세부비교['조수_Gap'] > 0]
조수감소세부 = 전산화번호_세부비교[전산화번호_세부비교['조수_Gap'] < 0]
요금증가세부 = 전산화번호_세부비교[전산화번호_세부비교['요금_Gap'] > 0]
요금감소세부 = 전산화번호_세부비교[전산화번호_세부비교['요금_Gap'] < 0]

신규세부 = 전산화번호_세부비교[전산화번호_세부비교['한전본부_y'].isnull()]
해지세부 = 전산화번호_세부비교[전산화번호_세부비교['한전본부_x'].isnull()]

In [16]:
조수증가요약 = pd.pivot_table(조수증가세부, index=['한전본부_x','지사명_x'],values=['조수_Gap'],aggfunc=sum,).reset_index()
조수감소요약 = pd.pivot_table(조수감소세부, index=['한전본부_x','지사명_x'],values=['조수_Gap'],aggfunc=sum,).reset_index()
요금증가요약 = pd.pivot_table(요금증가세부, index=['한전본부_x','지사명_x'],values=['요금_Gap'],aggfunc=sum,).reset_index()
요금감소요약 = pd.pivot_table(요금감소세부, index=['한전본부_x','지사명_x'],values=['요금_Gap'],aggfunc=sum,).reset_index()

신규요약 = pd.pivot_table(신규세부, index=['한전본부_x','지사명_x'],values=['총조수_x'],aggfunc=sum,).reset_index()
해지요약 = pd.pivot_table(해지세부, index=['한전본부_y','지사명_y'],values=['총조수_y'],aggfunc=sum,).reset_index()

In [27]:
조수증가요약.columns

Index(['한전본부', '지사명', '증가수'], dtype='object')

In [28]:
신규요약.columns

Index(['한전본부', '지사명', '신규수'], dtype='object')

In [29]:
해지요약.columns

Index(['한전본부', '지사명', '해지수'], dtype='object')

In [26]:
조수증가요약.rename(columns={'한전본부_x':'한전본부', '조수_Gap':'증가수', '지사명_x':'지사명'},inplace = True)
조수감소요약.rename(columns={'한전본부_x':'한전본부', '조수_Gap':'감소수', '지사명_x':'지사명'},inplace = True)
요금증가요약.rename(columns={'한전본부_x':'한전본부', '요금_Gap':'증가요금', '지사명_x':'지사명'},inplace = True)
요금감소요약.rename(columns={'한전본부_x':'한전본부', '요금_Gap':'감소요금', '지사명_x':'지사명'},inplace = True)

신규요약.rename(columns={'한전본부_x':'한전본부', '총조수_x':'신규수', '지사명_x':'지사명'},inplace = True)
해지요약.rename(columns={'한전본부_y':'한전본부', '총조수_y':'해지수', '지사명_y':'지사명'},inplace = True)

In [31]:
전산화번호_총괄요약 = pd.merge(조수증가요약,조수감소요약, on = ['한전본부','지사명'], how='left')

전산화번호_총괄요약 = pd.merge(전산화번호_총괄요약,요금증가요약, on = ['한전본부','지사명'], how='left' )
전산화번호_총괄요약 = pd.merge(전산화번호_총괄요약,요금감소요약, on = ['한전본부','지사명'], how='left' )
전산화번호_총괄요약 = pd.merge(전산화번호_총괄요약,신규요약, on = ['한전본부','지사명'], how='left' )
전산화번호_총괄요약 = pd.merge(전산화번호_총괄요약,해지요약, on = ['한전본부','지사명'], how='left' )

In [32]:
전산화번호_총괄요약

,한전본부,지사명,증가수,감소수,증가요금,감소요금,신규수,해지수
0,강원본부,강릉특별지사,382.0,-104.0,269565.0,-95261.0,116.0,21.0
1,강원본부,강원본부직할,120.0,-73.0,116715.0,-81324.0,159.0,27.0
2,강원본부,고성지사,82.0,-82.0,61746.0,-61746.0,8.0,8.0
3,강원본부,동해지사,250.0,-52.0,190133.0,-42169.0,33.0,20.0
4,강원본부,삼척지사,21.0,-24.0,15813.0,-18807.0,12.0,10.0
5,강원본부,속초지사,32.0,-49.0,24096.0,-53592.0,NaN,21.0
6,강원본부,양구지사,25.0,-8.0,15216.0,-6777.0,74.0,17.0
7,강원본부,양양지사,36.0,-36.0,27108.0,-27108.0,8.0,6.0
8,강원본부,영월지사,38.0,-19.0,14914.0,-54185.0,129.0,110.0
9,강원본부,원주지사,66.0,-51.0,49798.0,-109759.0,35.0,27.0


In [33]:
writer = pd.ExcelWriter('./가공엑셀자료/'+month_now+'전산화번호_요약정리.xlsx', engine='xlsxwriter')

In [34]:
전산화번호_총괄요약.to_excel(writer, sheet_name='총괄요약', startrow=1)
조수증가세부.to_excel(writer, sheet_name='조수증가', startrow=1)
조수감소세부.to_excel(writer, sheet_name='조수감소', startrow=1)
요금증가세부.to_excel(writer, sheet_name='요금증가', startrow=1)
요금감소세부.to_excel(writer, sheet_name='요금감소', startrow=1)
신규세부.to_excel(writer, sheet_name='신규세부내역', startrow=1)
해지세부.to_excel(writer, sheet_name='해지세부내역', startrow=1)

writer.save()

In [43]:
# 조수증가세부 = 전산화번호_세부비교[전산화번호_세부비교['조수_Gap'] > 0]
# 조수감소세부 = 전산화번호_세부비교[전산화번호_세부비교['조수_Gap'] < 0]
# 요금증가세부 = 전산화번호_세부비교[전산화번호_세부비교['요금_Gap'] > 0]
# 요금감소세부 = 전산화번호_세부비교[전산화번호_세부비교['요금_Gap'] < 0]

# 신규세부 = 전산화번호_세부비교[전산화번호_세부비교['한전본부_y'].isnull()]
# 해지세부 = 전산화번호_세부비교[전산화번호_세부비교['한전본부_x'].isnull()]